# Truss Stabilty Investigation by the Matrix Method of Analysis

The following truss is unstable.  Instead of determining that by finding a mechanism or by
use of the zero-load test, this section will setup a full matrix analysis.

![Figure](hextruss-irregular.svg)

Import some utilities from a library for statically determinate 2D truss analysis.

In [1]:
from sdtruss import sdtruss, SDTError

## Joint Coordinates
This section specifies the _x-y_ coordinates of each joint.  Each line of input specifies
the id of the joint and its _x_ and _y_ coordinates:

In [2]:
jc = [ ('a', 0, 0),
       ('b', 0, 4),
       ('c', 2, 8),
       ('d', 6, 8),
       ('e', 8, 4),
       ('f', 8, 0),
       ]

## Member Incidences
This section defines each member in the truss by specifying the ids of the joints
at either end of each member.  Each member is connected to exactly two joints.

In [3]:
mi = [ ('a', 'b',),   # there is a member from joint a to joint b, etc.
       ('b', 'c',),
       ('c', 'd',), 
       ('d', 'e',),
       ('e', 'f',),
       ('f', 'a',),
       ('b', 'e',),
       ('c', 'f',),
       ('a', 'd',),
       ]

## Reactions
This section specifies the reaction components.  Each line specifies one independent reaction
by naming the joint on which it impinges and the direction of the reaction.  Directions are given
as relative direction components.  For example, a direction component of '0,1' means a vertical
reaction (in the $y$-direction).  A direction component of '3,4' would specify an angle of 53.13 degrees to the $x$-axis ($\tan^{-1}{4\over3}$ - relative components of 3 in $x$ and 4
in $y$).

In [4]:
rf = [ ('a', 1, 0),             # reaction force dirns - horizontal at a
       ('a', 0, 1),             # vertical at a
       ('f', 0, 1),
       ]

## Joint Loads
Joint loads arfe specified by naming the joint, providing the magnitude, and direction components
in which it acts.

In [5]:
jl = [ ('c', 1, 1, 0),      # joint loads (j, p, dx, dy) - this is a unit load at c, in x-direction
       ]

## Analysis
Now we analyze by providing the above data to the library function.

In [6]:
try:
    sdtruss( jc, mi, rf, jl )
except SDTError as err:
    print('**** Error: {0}'.format(err))

**** Error: 'C' matrix is rank deficient.  Truss is unstable.


We see above the the rank-check of the matrix failed.  The rank of the coefficient matrix was less than
the number of columns and thus no solution was possible.

## Change the Geometry
Here we see a very small change made to the geometry, by moving joint _c_ up 1mm.

In [7]:
jc = [ ('a', 0, 0),
       ('b', 0, 4),
       ('c', 2, 8.001),
       ('d', 6, 8),
       ('e', 8, 4),
       ('f', 8, 0),
       ]

In [8]:
try:
    sdtruss( jc, mi, rf, jl )
except SDTError as err:
    print('**** Error: {0}'.format(err))

unknown  joint-i  joint-j        force
-------  -------  -------        -----
ab       a        b               1779 T
bc       b        c               1988 T
cd       c        d               2222 T
de       d        e               1988 T
ef       e        f               1778 T
fa       f        a               1334 T
be       b        e              889.1 C
cf       c        f               2224 C
ad       a        d               2222 C
Rax      a                          -1  @ 1,0
Ray      a                          -1  @ 0,1
Rfy      f                           1  @ 0,1


Mathematically, the truss is no longer unstable, but notice the very large member forces (compared
to the small applied load).  Forces were amplified by about a factor of 2000.  This demonstrates
that trusses that are unstable or nearly so will have very high member forces, leading to large distortions and geometry changes.  This is very undesireable in a civil engineering structure.